### CONCERNS
1. list of directions/positions are never emptied during an episode, which makes the simulation slower and slower after each timestep

### IDEAS
other possible features:
1. robot is fallen down or not
2. distance to border (& which border?)

time optimalization:
1. clear last item from history every 4 timesteps (we only use the current and previous state and the one before that)
2. interval of states to be interpreted: skip N frames before evaluation next state
3. Since the rewards are so sparse, maybe use Imitation learning instead of DQN --> we are "experts" since we know the tactic of the blue bot. we can use this to teach our bot how to defeat the other agent.
4. rewrite the random action function 

In [4]:
from VisualModule import AgentEnvironment
from DQN_Agent import NeurosmashAgent

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

import gym
import math
import random
from collections import namedtuple
from itertools import count
from PIL import Image
import os

from stopwatch import Stopwatch


Using TensorFlow backend.


In [5]:
model_output_dir = "output/model_output/"

if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

max_distance = 600
show_images = False
skip_frames = 15
state_size =  11 # agent_pos, enemy_pos, vec_agent, vec_enemy, rel_pos_enemy, done
action_size = 3
episode_count = 1000
batch_size = 32
size       = 768         # Please check the Updates section above for more details
timescale  = 10           # Please check the Updates section above for more details

In [6]:
agent = NeurosmashAgent(state_size = state_size, action_size = action_size) # action size: move in x or y direction, or do nothing

In [7]:
def compute_reward(standard_reward, distance):
    distance_reward = (max_distance-distance)/max_distance
    total_reward = (distance_reward + standard_reward)/ 2
    return total_reward

In [8]:
def direction(agent_path, enemy_path):
    A_X = (agent_path[-1] - np.array(agent_path[-2]))[0]
    A_Y = -(agent_path[-1] - np.array(agent_path[-2]))[1]
    E_X = (enemy_path[-1] - np.array(enemy_path[-2]))[0]
    E_Y = -(enemy_path[-1] - np.array(enemy_path[-2]))[1]
    return [A_X,A_Y],[E_X,E_Y]

def do_action(action, total_steps, eval_pic, environment):
    stopwatch = Stopwatch() 
    stopwatch.start()
    info, reward, agent_coord, enemy_coord, following_state = environment.actionLoop(action, 0, eval_pic)
    stopwatch.stop()
    #print(f"Total time for do action: {stopwatch.duration}")
    if len(environment.agent_path) < 2:
        distance = 500 # Initial distance, only for initialisation
        agent_direction = [1,0] # By definition of facing each other
        enemy_direction = [-1,0]
    else:
        distance = np.sqrt(np.square(np.array(list(np.array(agent_coord)- np.array(enemy_coord))).sum(axis=0)))
        # Extract all variables 
        agent_direction, enemy_direction  = direction(environment.agent_path, environment.enemy_path)
    
    complete_reward = compute_reward(reward, distance)

    rel_pos_enemy = np.array(enemy_coord) - np.array(agent_coord)
    return info, complete_reward, np.array(agent_coord), np.array(enemy_coord), agent_direction, enemy_direction, distance, rel_pos_enemy, following_state


In [9]:
env = AgentEnvironment(size=size, timescale=timescale)
def init_environment(agent_here):
    #env = AgentEnvironment(size=size, timescale=timescale)
    info, reward, state = env.reset() 
  
    agent_trajectories = []
    enemy_trajectories = []
    
    small_init_state = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    small_init_state = np.reshape(small_init_state, [1, state_size])
 
    
    #for i in range(3):
    print(f"act called from init: {small_init_state}")

    action = agent_here.act(small_init_state) # get next action
    # action = 3 (if above does not work)
    
    #pre_state_img = np.flip(np.array(state).reshape(3,256,256).transpose(1,2,0),0)
    #step_number_now = i+1
    info, complete_reward, agent_pos, enemy_pos, agent_direction, enemy_direction, distance, relative_pos_enemy, next_state = do_action(action, 1, True, env)  
    
    #post_state_img = np.flip(np.array(next_state).reshape(3,256,256).transpose(1,2,0),0)

    #agent_pos, enemy_pos = env_feat.coord(pre_state_img, post_state_img)
    agent_trajectories.append(list(agent_pos))
    enemy_trajectories.append(list(enemy_pos))
        
    return info, complete_reward, next_state, agent_trajectories, enemy_trajectories, agent_direction, relative_pos_enemy, enemy_direction, env

In [ ]:
complete_rewards = []
for e in range(episode_count):
    status, complete_reward, next_state, agent_trajectories, enemy_trajectories, agent_dir, relative_pos_enemy, enemy_dir, environment = init_environment(agent)
    small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[-1][0], enemy_trajectories[-1][1], agent_dir[0], agent_dir[1], relative_pos_enemy[0], relative_pos_enemy[1], enemy_dir[0], enemy_dir[1], 0]#"agent direction", "relative position enemy", "enemy direction" ]
    small_state = np.reshape(small_state, [1, state_size])

    done = 0
    total_reward = 0
    total_timesteps = 1
    distances = []
    evaluate_frame = False

    while done == False:
        if (total_timesteps % skip_frames == 0) or (total_timesteps % skip_frames == skip_frames-1):
            evaluate_frame = True
        else:
            evaluate_frame = False
        
        print(f"act called from main loop: {small_state}")
        action = agent.act(small_state) #step(info, reward, state)
        #print(f"agent chooses action: {action}")
        stopwatch = Stopwatch() 
        stopwatch.start()
        status, complete_reward, agent_pos, enemy_pos, agent_dir, enemy_dir, distance, enemy_pos_rel, next_state = do_action(action, total_timesteps, evaluate_frame, environment)   
        stopwatch.stop()
        #print(f"Total time for one step: {stopwatch.duration}")
        
        total_reward += complete_reward

        if status == 1:
            print(f"Game is finished, \n your final reward is: {total_reward}, duration was {total_timesteps} timesteps")
            done = 1
        
        agent_trajectories.append(list(agent_pos))
        enemy_trajectories.append(list(enemy_pos))
        distances.append(distance)
        
        done_list = [done]
        next_small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[1][0], enemy_trajectories[1][1], agent_dir[0], agent_dir[1], enemy_pos_rel[0], enemy_pos_rel[1], enemy_dir[0], enemy_dir[1], done]  
    
        next_small_state = np.reshape(next_small_state, [1, state_size]) # why?
        small_state = np.reshape(small_state, [1, state_size])
        
        if (total_timesteps % skip_frames == 0):
            print(f"agent will remember: {small_state}")
            agent.remember(small_state, action, complete_reward, next_small_state, list(done_list))
        
        small_state = next_small_state # new small state
        total_timesteps += 1
        
    complete_rewards.append(total_reward)

    if len(agent.memory) > batch_size:
        agent.train(batch_size)
        print("train")

    if e % 50 == 0:
        agent.save(model_output_dir + "weights_"+ '{:04d}'.format(e) + ".hdf5")
            
    
    

act called from init: [[0 0 0 0 0 0 0 0 0 0 0]]
act called from main loop: [[137 187 652 196   1   0 515   9  -1   0   0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[167 182   0   0 

agent will remember: [[306 207   0   0  -5 -60  26 -79 -39  22   0]]
act called from main loop: [[306 207   0   0   0   0  30 -74   4  -5   0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loop: [[ 0  0  0  0  0  0  0  0  4 -5  0]]
act called from main loo

act called from main loop: [[513 193   0   0  44  -5 -26   3  48   7   0]]
agent will remember: [[513 193   0   0  44  -5 -26   3  48   7   0]]
act called from main loop: [[514 195   0   0   1  -2 -22   3   5  -2   0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called from main loop: [[ 0  0  0  0  1 -2  0  0  5 -2  0]]
act called fr

act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[ 0  0  0  0  3  1  0  0 -2 -4  0]]
act called from main loop: [[303 163   0   0  27  15 171  16 -42  12   0]]
agent will remember: [[303 163   0   0  27  15 171  16 -42  12   0]]
act called from main loop: [[304 161   0   0   1   2 166  19  -4  -1   0]]
act called from main loop: [[ 0  0  0  0  1  2  0  0 -4 -1  0]]
act called from main loop: [[ 0  0  0  0  1  2  0  0 -4 -1  0]]
act called fr

act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 0  0  0  0  1  1  0  0 -1 -8  0]]
act called from main loop: [[ 446   82    0    0   16   19 -138   58   57   16    0]]
agent will remember: [[ 446   82    0    0   16   19 -138   58   57   16    0]]
act called from main loop: [[ 448   79    0    0    2    3 -139   

...

act called from main loop: [[ 715  150    0    0   53   -9 -202  105   32   22    0]]
agent will remember: [[ 715  150    0    0   53   -9 -202  105   32   22    0]]
act called from main loop: [[ 717  150    0    0    2    0 -201  112    3   -7    0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main loop: [[ 0  0  0  0  2  0  0  0  3 -7  0]]
act called from main

act called from init: [[0 0 0 0 0 0 0 0 0 0 0]]
act called from main loop: [[137 187 667 196   1   0 530   9  -1   0   0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1

act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from main loop: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  2  1  0  0 -2  0  0]]
act called from 

act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
state that the model will use to predict action: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
state that the model will use to predict action: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
state that the model will use to predict action: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
state that the model will use to predict action: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
state that the model will use to predict action: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8  0]]
act called from main loop: [[ 0  0  0  0  0 -1  0  0  0 -8

act called from main loop: [[137 187 653 196   1   0 516   9  -1   0   0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will use to predict action: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
act called from main loop: [[ 0  0  0  0  1  0  0  0 -1  0  0]]
state that the model will u